# **Resume Parsing**

I will use 'Tika' to parse the resumes and extract necessary data from them. Spacy will be my primary tool to extract data and index it. I will then use Spacy's Matcher to match it with specific indices of my 'keywords'. I have to include some keywords which are not required, because the data in resumes has been presented in such a format.  

In [1]:
# !pip install tika

In [2]:
parsed_content = {}

Warning: The following cell should not be run more than once per notebook's run instance. It will shift to the next available proper nouns if run multiple times. 

In [3]:
import spacy
import re
def tparser(text):
    nlp = spacy.load('en_core_web_sm')
    from spacy.matcher import Matcher
    matcher = Matcher(nlp.vocab)

    def extract_categ(text):
        nlp_text = nlp(text)
        
        
        pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        
        matcher.add('NAME', [pattern], on_match = None)
        
        matches = matcher(nlp_text)
        
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            return span.text

    name = extract_categ(text)

    parsed_content['Category'] =  name
    Keywords = ["education",
                "summary",
                "accomplishments",
                "executive profile",
                "professional profile",
                "personal profile",
                "work background",
                "academic profile",
                "other activities",
                "qualifications",
                "experience",
                "interests",
                "skills",
                "achievements",
                "publications",
                "publication",
                "certifications",
                "workshops",
                "projects",
                "internships",
                "trainings",
                "hobbies",
                "overview",
                "objective",
                "position of responsibility",
                "jobs"
            ]
   
    text = text.replace("\n"," ")
    text = text.replace("[^a-zA-Z0-9]", " ");  
    re.sub('\W+','', text)
    text = text.lower()
    
    content = {}
    indices = []
    keys = []
    for key in Keywords:
        try:
            content[key] = text[text.index(key) + len(key):]
            indices.append(text.index(key))
            keys.append(key)
        except:
            pass
        
    #Sorting the indices
    zipped_lists = zip(indices, keys)
    sorted_pairs = sorted(zipped_lists)
    sorted_pairs

    tuples = zip(*sorted_pairs)
    indices, keys = [ list(tuple) for tuple in  tuples]
    keys
    #Keeping the required content and removing the redundant part
    content = []
    for idx in range(len(indices)):
        if idx != len(indices)-1:
            content.append(text[indices[idx]: indices[idx+1]])
        else:
            content.append(text[indices[idx]: ])
    for i in range(len(indices)):
        parsed_content[keys[i]] = content[i]  
    return parsed_content

In [24]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tika import parser
import os

folder_path = 'C:/Users/abhyu/Documents/projexts/capital/data/data/INFORMATION-TECHNOLOGY/'
data = {'Category':[''], 'Skills':[''], 'Education':[''], 'Experience':['']}
# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)  # Full file path
        file_data = parser.from_file(file_path)
        text = file_data['content']
        parsed_content = tparser(text)
        data['Category'].append(parsed_content.get('Category', ''))
        data['Skills'].append(parsed_content.get('skills', ''))
        data['Education'].append(parsed_content.get('education', ''))
        data['Experience'].append(parsed_content.get('experience', ''))


df = pd.DataFrame(data)
df

Category  \
0                             
1         Versatile Systems   
2      Professional Summary   
3                 keynote Â   
4         Project trackingÂ   
..                      ...   
116         Career Overview   
117          DIRECTOR HUMAN   
118      Christopher Townes   
119    Professional Summary   
120  INFORMATION TECHNOLOGY   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Skills  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [25]:
df.to_csv('parsed_res_it.csv')